In [5]:
from pyshacl import validate
from rdflib import Graph

def validate_rdf_with_shacl(rdf_file, shacl_file):
    """
    Validate an RDF file against a SHACL file.

    :param rdf_file: Path to the RDF file to validate.
    :param shacl_file: Path to the SHACL file with constraints.
    :return: Tuple (conforms, report) where conforms is a boolean indicating
             if the RDF conforms to the SHACL constraints, and report is a string report.
    """
    # Load RDF and SHACL files into RDFLib Graphs
    rdf_graph = Graph()
    shacl_graph = Graph()

    rdf_graph.parse(rdf_file, format="turtle")
    shacl_graph.parse(shacl_file, format="turtle")

    # Validate the RDF graph against the SHACL graph
    conforms, report_graph, report_text = validate(
        data_graph=rdf_graph,
        shacl_graph=shacl_graph,
        inference="rdfs",
        abort_on_error=False,
        meta_shacl=False,
        debug=False
    )

    return conforms, report_text

# File paths
rdf_file = "ability_kg.ttl"
shacl_file = "ability_shapes.ttl"

# Perform validation
conforms, report = validate_rdf_with_shacl(rdf_file, shacl_file)

# Print results
if conforms:
    print("The RDF file conforms to the SHACL constraints.")
else:
    print("The RDF file does NOT conform to the SHACL constraints.")

print("Validation Report:")
print(report)


Usage of abort_on_error is deprecated. Use abort_on_first instead.


The RDF file conforms to the SHACL constraints.
Validation Report:
Validation Report
Conforms: True



In [12]:
import re

def replace_subjects_with_prefix_and_add_sameas_start(input_file, output_file):
    prefix = """@prefix ability: <http://example.org/pokemon/ability/> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix bulbapedia: <https://bulbapedia.bulbagarden.net/wiki/> .
@prefix pkmn: <http://example.org/pokemon/> .
@prefix schema: <http://schema.org/> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .\n"""
    
    subject_pattern = r"<http://example.org/pokemon/ability/([a-zA-Z0-9_]+)>"

    with open(input_file, 'r', encoding='utf-8') as infile:
        lines = infile.readlines()

    updated_lines = []
    prefix_added = False
    ability_name = None

    for line in lines:
        # Add the prefix at the beginning of the file if not already added
        if not prefix_added:
            updated_lines.append(prefix)
            prefix_added = True

        # Replace subjects matching the pattern with the compact form
        if match := re.match(subject_pattern, line):
            ability_name = match.group(1)  # Capture the ability name
            updated_line = re.sub(subject_pattern, r"ability:\1", line)
            # Add owl:sameAs immediately after the subject declaration
            sameas_line = f"    owl:sameAs bulbapedia:{ability_name} ;\n"
            updated_lines.append(updated_line)
            updated_lines.append(sameas_line)
        else:
            updated_line = line
            updated_lines.append(updated_line)

    with open(output_file, 'w', encoding='utf-8') as outfile:
        outfile.writelines(updated_lines)

# Update the input and output file paths
input_file = "ability_kg.ttl"
output_file = "ability_kg.ttl"

replace_subjects_with_prefix_and_add_sameas_start(input_file, output_file)
